# Bayes - Spam Filtering
# Introduction

Throughout this laboratory you will be working with an SMS database. The text messages in this database are either spam or ham (not spam). You are to classify them using Bayesian probabilities and will therefore create a Naive Bayes classifier.

Serving as guide for this laboratory will be the code comments and the following formula(s):

**P(A∣B) = (P(A) * P(B∣A)) / P(B)**

P(A∣B1, B2, ..., Bn) = (P(B1|A) * P(B2∣A) * ... * P(Bn|A) * P(A)) / (P(B1) * P(B2) * ... * P(Bn)) 

where:

P(A) = The probability of A occurring

P(B) = The probability of B occurring

P(A∣B) =The probability of A given B

P(B∣A) = The probability of B given A

E. g. P(Ham|w1, w2, w3) is the probability that a message is ham given that it contains the words w1, w2 and w3.
​	


# Warming up


In [188]:
# Warm-up
# The function must return P(A|B)
# You are given P(A), P(B) and P(B|A) as parameters
def prob_A_if_B(prob_A, prob_B, prob_B_if_A):
    raise NotImplementedError

prob_rain = 0.1 # Probability that it rains throughout any given day
prob_morning_cloud = 0.4 # Probability of having clouds in the morning of any given day
prob_morning_cloud_if_rain = 0.5 # Probability that the morning was cloudy in any rainy day

#Calculate probability of raining if the morning was cloudy
print(prob_A_if_B(prob_rain, prob_morning_cloud, prob_morning_cloud_if_rain)) # Should be 0.125

NotImplementedError: ignored

# Read the data from file
You will be reading a set of 5572 messages into a pandas dataframe

In [189]:
import pandas as pd
import re

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t',
header=None, names=['Label', 'SMS'])
print(sms_spam.shape)
print(sms_spam.head())

(5572, 2)
  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


# Split data into train and test sets

In [190]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape) # We train on 4458 messages
print(test_set.shape) # At the end we'll test our classifier on 1114 messages

(4458, 2)
(1114, 2)


# Clean data

In [191]:
# Redundant, but done twice just in case you run this cell multiple times (avoids an error)
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

# Normalize and clean data (lowercase, remove punctuation)
training_set['Label'].value_counts(normalize=True)
test_set['Label'].value_counts(normalize=True)

# Before cleaning
print("Before cleaning:")
print(training_set.head(3))

# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace(
   '\W', ' ') # Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()

print("\n After cleaning:")
print(training_set.head(3))

# Split string into words
training_set['SMS'] = training_set['SMS'].str.split()

print("\n After splitting into words:")
print(training_set.head(3))

Before cleaning:
  Label                                            SMS
0   ham                   Yep, by the pretty sculpture
1   ham  Yes, princess. Are you going to make me moan?
2   ham                     Welp apparently he retired

 After cleaning:
  Label                                            SMS
0   ham                   yep  by the pretty sculpture
1   ham  yes  princess  are you going to make me moan 
2   ham                     welp apparently he retired

 After splitting into words:
  Label                                                SMS
0   ham                  [yep, by, the, pretty, sculpture]
1   ham  [yes, princess, are, you, going, to, make, me,...
2   ham                    [welp, apparently, he, retired]


# Create dictionary of word occurences

In [192]:
# Save a list of the words found in all of the training messages
vocabulary = []
for sms in training_set['SMS']:
   for word in sms:
      vocabulary.append(word)

# Remove duplicates
vocabulary = list(set(vocabulary))

print(vocabulary)

['suprman', 'macs', 'astronomer', 'bari', 'usb', 'f4q', 'sextextuk', 'exorcist', 'radiator', 'ur', 'boltblue', 'm100', 'velachery', 'concentrating', 'ing', 'mrng', 'gets', 'tomorro', 'actual', 'newspapers', 'db', 'banned', 'important', 'gent', 'familiar', 'titles', 'my', 'phrase', 'gym', 'tirunelvali', 'unsub', 'cook', 'warned', '0844', '87239', 'leafcutter', 'onam', 'celeb', 'betta', 'revision', '40533', 'pei', 'pobox', 'loss', 'throw', 'deleted', 'shaping', 'cuddling', 'long', 'follow', 'tv', 'flight', 'bffs', '2006', 'slide', 'answer', 'medicine', 'hurts', 'drinks', 'cme', 'responding', 'weirdest', 'grahmbell', 'east', 'rose', 'deposit', 'la3', 'bcs', 'letter', 'replies', 'wonderful', 'oyster', 'sao', 'dysentry', 'how', 'gv', 'brighten', 'ignoring', 'loose', 'messaged', 'cheers', 'outfor', 'dawns', 'i', 'ros', 'twelve', 'violated', 'arrive', 'onion', 'sankatmochan', 'probthat', 'okies', 'illspeak', 'takes', 'nokias', '2004', 'discussed', 'afternoon', 'small', '38', 'understand', 'fr

In [193]:
# Create an empty dictionary where each key is a (unique) word, each value is a list of num_training_messages zeros
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

# Fill the dictionary by adding to each word the number of times it appears in each message 
# e.g. word_counts_per_sms['ur'][5] = 2 means that the word "ur" appears twice in the 5th message of the dataset
for index, sms in enumerate(training_set['SMS']):
   for word in sms:
      word_counts_per_sms[word][index] += 1

# Transform everything back to a pandas dataframe
word_counts = pd.DataFrame(word_counts_per_sms)

# Add the label and SMS to the data 
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,suprman,macs,astronomer,bari,usb,f4q,sextextuk,exorcist,radiator,ur,boltblue,m100,velachery,concentrating,ing,mrng,gets,tomorro,actual,newspapers,db,banned,important,gent,familiar,titles,my,phrase,gym,tirunelvali,unsub,cook,warned,0844,87239,leafcutter,onam,celeb,...,locations,eye,priya,facilities,urfeeling,woke,peak,ticket,inconvenient,understanding,bruv,abusers,eerie,simulate,garbage,vale,coincidence,welcomes,gbp4,awesome,gained,stone,plate,unintentionally,festival,hl,wipe,fiend,tron,starving,accomodations,obviously,eveb,fool,dec,left,kisses,keluviri,blank,habba
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Number and probability of spam, ham messages (to be implemented)
# Calculate individual word probabilities for both ham & spam

In [194]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# You have the spam_messages, ham_messages and all the training set messages
# Calculate the probability of a message being spam (p_spam) and probability of a message being ham (p_ham)
# store the values in p_spam and p_ham (both numbers should be a decimal number between 0 and 1)
# P(Spam) and P(Ham)

# TO BE IMPLEMENTED 
p_spam = 0.5 # Replace the 0.5 with the correct code to compute p_spam
p_ham = 0.5  # Replace the 0.5 with the correct code to compute p_ham

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing - add 1 (alpha) to each word count so as to avoid probabilities equal to 0
alpha = 1

print("p_spam =", p_spam)
print("p_ham =", p_ham)

p_spam = 0.5
p_ham = 0.5


In [195]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate and store parameters (probabilities) for each word if message is spam and if message is ham
for word in vocabulary:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham

# Probability of word existing given that message is ham 
print("ham word probabilities:", parameters_ham, '\n')

# Probability of word existing given that message is spam 
print("spam word probabilities:", parameters_spam)


ham word probabilities: {'suprman': 1.537988311288834e-05, 'macs': 3.075976622577668e-05, 'astronomer': 3.075976622577668e-05, 'bari': 3.075976622577668e-05, 'usb': 4.6139649338665025e-05, 'f4q': 1.537988311288834e-05, 'sextextuk': 1.537988311288834e-05, 'exorcist': 1.537988311288834e-05, 'radiator': 3.075976622577668e-05, 'ur': 0.0030913565056905567, 'boltblue': 1.537988311288834e-05, 'm100': 3.075976622577668e-05, 'velachery': 3.075976622577668e-05, 'concentrating': 3.075976622577668e-05, 'ing': 4.6139649338665025e-05, 'mrng': 0.00023069824669332513, 'gets': 0.0001845585973546601, 'tomorro': 3.075976622577668e-05, 'actual': 3.075976622577668e-05, 'newspapers': 3.075976622577668e-05, 'db': 3.075976622577668e-05, 'banned': 1.537988311288834e-05, 'important': 0.00016917871424177176, 'gent': 1.537988311288834e-05, 'familiar': 3.075976622577668e-05, 'titles': 1.537988311288834e-05, 'my': 0.009197170101507229, 'phrase': 3.075976622577668e-05, 'gym': 0.00015379883112888343, 'tirunelvali': 4

# Classify message function (to be implemented)


In [196]:
# Classify message into spam or ham
# Function must return either 'ham', 'spam' or 'unknown' 
# You can access the variables p_spam, p_ham and parameters_spam from within the function, you will need them for your classification
# Remember the Bayes Formula. We are trying to calculate P(Ham|w1, w2, ..., wn) and P(Spam|w1, w2, ..., wn) where w1, w2, ..., wn are our words.
def classify_message(message):
    '''
    message: a string
    '''
    # Remove punctuation from the message, change message to lowercase and then split it into words
    # (message becomes a list of strings, each string is a word)
    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    # Delete this print() when writing the function. It is here just to point out the variables you'll be needing.
    print(message, p_spam, p_ham, parameters_spam['ur'])

    raise NotImplementedError

    


# Predictions and accuracy

In [197]:
# See some predictions of your classifier on the test dataset
test_set['predicted'] = test_set['SMS'].apply(classify_message)
test_set.head(10)

['later', 'i', 'guess', 'i', 'needa', 'do', 'mcat', 'study', 'too'] 0.5 0.5 0.005092935184782136


NotImplementedError: ignored

In [ ]:
# Test your own custom-made messages. Replace the ones below with whatever you want.
my_message = 'Hello, how are you?' 
my_spam_message = "Call fast BIG reward" 

print(classify_message(my_message))
print(classify_message(my_spam_message))

In [ ]:
correct = 0
total = test_set.shape[0]

# Iterate through the test_set, check how many predictions were correct
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
         correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)

# If accuracy is above 0.98, your code is correct
print('Accuracy:', correct/total)

# Create tests

In [ ]:
def test_0_p_spam_ham():
    if p_spam < 0.11 or p_spam > 0.14:
        return False
    if p_ham > 0.88 or p_ham < 0.84:
        return False
    return True

def test_1_ham():
    pred_test_list = [classify_message(test_set['SMS'][100]), classify_message(test_set['SMS'][300]), classify_message(test_set['SMS'][400]), classify_message(test_set['SMS'][777])]
    corr_test_list = ['ham', 'ham', 'ham', 'ham']                                                                                                                            
    return pred_test_list == corr_test_list

def test_2_spam():
    pred_test_list = []
    pred_test_list.append(classify_message(test_set['SMS'][500]))
    pred_test_list.append(classify_message(test_set['SMS'][2]))
    pred_test_list.append(classify_message(test_set['SMS'][28]))
    pred_test_list.append(classify_message(test_set['SMS'][121]))
    corr_test_list = ['spam', 'spam', 'spam', 'spam']  

    return pred_test_list == corr_test_list

def test_3_combined():
    pred_test_list = []
    pred_test_list.append(classify_message(test_set['SMS'][111]))
    pred_test_list.append(classify_message(test_set['SMS'][222]))
    pred_test_list.append(classify_message(test_set['SMS'][333]))
    pred_test_list.append(classify_message(test_set['SMS'][444]))
    pred_test_list.append(classify_message(test_set['SMS'][555]))
    pred_test_list.append(classify_message(test_set['SMS'][556]))
    pred_test_list.append(classify_message(test_set['SMS'][777]))
    pred_test_list.append(classify_message(test_set['SMS'][678]))
    corr_test_list = ['ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham']

    return pred_test_list == corr_test_list

# Run tests

In [ ]:
def run_tests():
    test_names = ['test_0_p_spam_ham', 'test_1_ham', 'test_2_spam', 'test_3_combined']
    tests_passed = 0
    for i in range(4):
        if eval(test_names[i] + '()') is True:
            print("Test " + str(i + 1) + '/4 passed')
            tests_passed += 1
        else:
            print("Test " + str(i + 1) + '/4 failed')
    if tests_passed == 4:
        print("\nAll tests have passed")
    else:
        print(str(tests_passed) + "/4 tests passed in total")
run_tests()